Assignment from Dr. Paúl Pauca's CSC375: Deep Learning & Neural Network Class. All credit to him.

# Multi-class classification via unsupervised methods
In this project, we experiment with unsupervised machine learning methods for solving a multi-class classification problem. Unsupervised methods are computational methods that classify or cluster data by discovering hidden patterns or structure within the data itself, without the help of labeled training data. Unsupervised methods are widely used in data science applications in a variety of fields (medicine, finance, e-commerce, etc.) where labeled training data might be hard to obtain.

Here, we are going to apply a well-known unsupervised technique called hierarchical clustering and k-means clustering to the MNIST dataset multi-class classification problem. All the code and experimentation will be done in this Jupyter notebook.

# Multi-class classification: the MNIST dataset

The MNIST dataset consists of 70,000 gray-scale images (samples) of hand-written digits 0 through 9. The multi-class classification problem consists of classifying each sample accurately as belonging to one of ten classes. For the purpose of this assignment, we will only use the labels provided for each sample in the dataset for determining classification accuracy of our results.

# Part 1: Hierarchical Clustering

Hierarchical clustering is a well-known technique used heavily in data science applications. A cluster is a subset of the data which are deemed to be similar (in some sense). Clustering methods divide a dataset into clusters in such a way that the members of the same cluster are more similar to each other than to members of other clusters. Hierarchical clustering can be either top-down or bottom-up. The top-down (or divisive) approach starts with all the data in a single cluster and then successively divides clusters into smaller ones as it goes down the hierarchy. In contrast, the bottom-up (or agglomerative) approach starts with each sample in its own cluster and then pairs of clusters are merged as it goes up the hierarchy.



In [ ]:
#Import required libraries
import numpy as np
import pandas as pd

import scipy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

from pylab import rcParams
import seaborn as sb
import matplotlib.pyplot as plt

import sklearn
from sklearn.cluster import AgglomerativeClustering
import sklearn.metrics as sm
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Get required data
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#Create a batch of training data of size 10,000
X_train = np.asarray(train_images.reshape(train_images.shape[0],-1))
X_batch= X_train[0:10000,:]

#Create a batch of the corresponding correct labels of size 10,000
y = train_labels
y_prec = train_labels[0:10000]

#Ensure this subset is a roughly accurate representation of the whole dataset
print(np.asarray(np.unique(y_prec, return_counts=True)))


11493376/11490434 [==============================] - 0s 0us/step
[[   0    1    2    3    4    5    6    7    8    9]
 [1001 1127  991 1032  980  863 1014 1070  944  978]]


In [ ]:
#mapCluster method to determine the correct cluster to compute accuracy

def mapCluster(y_clusters, y, numClusters):
  y_pred = np.zeros(y_clusters.shape).astype(np.int32)
  mapOfCluster = np.zeros(numClusters).astype(np.int32) * -1
  #Map clusters to digits in mapOfClusters by choosing the digit most often associated with each cluster
  for cluster in range(0, numClusters):
    #Count number of true labels for each member in this cluster
    count = np.bincount(y[y_clusters == cluster], minlength=10)
    #Map cluster to the true label of most cluster members
    #Pick the index of the first max valuein count if there are more than max values
    mapOfCluster[cluster] = np.argmax(count)

    # print(cluster, "map to", y_pred[cluster], ", counts: ", count)

  #Now use mapOfCluster to map each y_cluster to predicted class
  y_pred = mapOfCluster[y_clusters]
  #This is the loop version of the line above
  # for i in range(0, len(y_clusters)):
    #     y_pred[i] = mapOfCluster[y_clusters[i]]

  return y_pred

# Parameters Used

All combination of affinity and linkage will be tried to determine the best parameters. The available options are “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed” for affinity and “ward”, “complete”, “average”, “single” for linkage. Note: Ward linkage can only be used in combination with euclidean affinity.

In [ ]:
#Try euclidean ward

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='euclidean', linkage='ward')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)


0.6958


array([[ 979,    1,    1,    6,    2,    0,    9,    0,    3,    0],
       [   0, 1088,   12,    5,    1,    0,    0,   19,    2,    0],
       [  13,    2,  885,    9,    2,    0,    1,   66,   13,    0],
       [   1,    0,    9,  961,    5,    0,    1,   32,   23,    0],
       [   0,    1,    2,    0,  470,    0,   16,  491,    0,    0],
       [   2,    0,    3,  277,    3,    0,    9,  323,  246,    0],
       [  10,    1,    1,   13,    2,    0,  978,    1,    8,    0],
       [   0,    3,    2,    7,   40,    0,    0, 1018,    0,    0],
       [   2,    4,    3,  279,   11,    0,    6,   60,  579,    0],
       [   1,    1,    2,   20,  395,    0,    2,  555,    2,    0]])

In [ ]:
#Try using Euclidean Ward with all data to see if results change

for i in range(6):
  #Create a batch of training data of size 10,000 from first 10,000
  X_train = np.asarray(train_images.reshape(train_images.shape[0],-1))
  X_batch= X_train[(i * 10000):((i*10000) +10000),:]

  #Create a batch of the corresponding correct labels of size 10,000
  y = train_labels
  y_prec = train_labels[(i * 10000):((i*10000) +10000)]

  # Compute hierarchical clustering
  numClusters = 10
  Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='euclidean', linkage='ward')
  Hclustering.fit(X_batch)

  # map the clusters to their digit of maximum frequency in the MNIST dataset
  clusters = Hclustering.labels_
  y_pred = mapCluster(clusters, y_prec, numClusters)
  print('Subset [', i*10000, ':', (i*10000+10000), ']: ', sm.accuracy_score(y_pred, y_prec))

Subset [ 0 : 10000 ]:  0.6958
Subset [ 10000 : 20000 ]:  0.6378
Subset [ 20000 : 30000 ]:  0.6378
Subset [ 30000 : 40000 ]:  0.6352
Subset [ 40000 : 50000 ]:  0.6388
Subset [ 50000 : 60000 ]:  0.6949


In [ ]:
#Try euclidean/L2 average

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='euclidean', linkage='average')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.2297


array([[ 960,   31,    0,    0,    0,    0,    9,    0,    0,    1],
       [   0, 1126,    0,    0,    0,    0,    0,    0,    1,    0],
       [   7,  978,    0,    0,    0,    0,    3,    0,    2,    1],
       [   6,  957,    0,   68,    0,    0,    0,    0,    0,    1],
       [   0,  959,    0,    0,    3,    0,   18,    0,    0,    0],
       [   4,  833,    0,   23,    0,    0,    2,    0,    1,    0],
       [   7,  877,    0,    5,    0,    0,  125,    0,    0,    0],
       [   0, 1068,    0,    0,    0,    0,    1,    0,    0,    1],
       [  14,  919,    0,    2,    0,    0,    0,    0,    9,    0],
       [   5,  958,    0,    1,    0,    0,    8,    0,    0,    6]])

In [ ]:
#Try euclidean/L2 complete

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='euclidean', linkage='complete')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.4109


array([[ 860,   36,    8,   37,   12,    0,   48,    0,    0,    0],
       [   0, 1117,    0,    1,    0,    0,    4,    0,    5,    0],
       [  21,   93,  546,  154,    6,    0,  115,    0,   56,    0],
       [  31,  234,   36,  586,    4,    0,  127,    0,   14,    0],
       [   0,  475,    8,    2,  491,    0,    3,    0,    1,    0],
       [  15,  493,    1,  237,    6,    0,  108,    0,    3,    0],
       [   8,  432,  178,   12,  156,    0,  228,    0,    0,    0],
       [   0,  824,    2,    8,  212,    0,    4,    0,   20,    0],
       [  10,  267,    3,  305,   22,    0,   56,    0,  281,    0],
       [   5,  664,    0,   17,  284,    0,    4,    0,    4,    0]])

In [ ]:
#Try euclidean/L2 single

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='euclidean', linkage='single')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.1136


array([[   0, 1001,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1127,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  991,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1030,    0,    2,    0,    0,    0,    0,    0,    0],
       [   0,  979,    0,    0,    1,    0,    0,    0,    0,    0],
       [   0,  862,    0,    0,    0,    1,    0,    0,    0,    0],
       [   0, 1014,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1070,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  941,    0,    0,    0,    0,    0,    0,    3,    0],
       [   0,  976,    0,    0,    0,    0,    0,    0,    0,    2]])

In [ ]:
#Try manhattan/L1 average

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='manhattan', linkage='average')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.2287


array([[ 963,   34,    0,    0,    0,    0,    4,    0,    0,    0],
       [   0, 1126,    0,    0,    0,    0,    0,    0,    1,    0],
       [   5,  978,    1,    0,    0,    0,    5,    0,    2,    0],
       [  10,  939,    0,   82,    0,    0,    1,    0,    0,    0],
       [   0,  976,    0,    0,    0,    0,    4,    0,    0,    0],
       [  30,  797,    0,   36,    0,    0,    0,    0,    0,    0],
       [  13,  897,    0,    0,    0,    0,  104,    0,    0,    0],
       [   0, 1070,    0,    0,    0,    0,    0,    0,    0,    0],
       [  17,  914,    1,    1,    0,    0,    1,    0,   10,    0],
       [   6,  969,    1,    1,    0,    0,    0,    0,    0,    1]])

In [ ]:
#Try manhattan/L1 complete

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='manhattan', linkage='complete')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.3649


array([[667,  41,   0, 274,   6,   0,  13,   0,   0,   0],
       [  3, 939, 181,   4,   0,   0,   0,   0,   0,   0],
       [111, 137, 613,  30,   3,   0,  97,   0,   0,   0],
       [  8, 445,  18, 556,   1,   0,   4,   0,   0,   0],
       [  0, 638,   7,   5, 328,   0,   2,   0,   0,   0],
       [ 25, 557,   1, 268,  11,   0,   1,   0,   0,   0],
       [  6, 119,   0, 278,  65,   0, 546,   0,   0,   0],
       [ 74, 897,   2,   3,  91,   0,   3,   0,   0,   0],
       [ 90, 582,   0, 251,  12,   0,   9,   0,   0,   0],
       [  3, 805,   0,  10, 158,   0,   2,   0,   0,   0]])

In [ ]:
#Try manhattan/L1 single

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='manhattan', linkage='single')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.1136


array([[   0, 1001,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1127,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  991,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1030,    0,    2,    0,    0,    0,    0,    0,    0],
       [   0,  980,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  862,    0,    0,    0,    1,    0,    0,    0,    0],
       [   0, 1014,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1070,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  939,    0,    0,    0,    0,    0,    0,    5,    0],
       [   0,  977,    0,    0,    0,    0,    0,    0,    0,    1]])

In [ ]:
#Try cosine average

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='cosine', linkage='average')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.2222


array([[   2,    1,    0,    0,    0,    0,    6,  992,    0,    0],
       [   0, 1108,    0,    0,    0,    0,    0,   19,    0,    0],
       [   0,   31,    9,    0,    0,    0,    2,  949,    0,    0],
       [   1,   11,    3,    1,    0,    0,    0, 1016,    0,    0],
       [   0,   20,    0,    0,   11,    0,    8,  941,    0,    0],
       [   0,  345,    0,    1,    0,    0,    1,  516,    0,    0],
       [   0,   12,    0,    0,    0,    0,   36,  966,    0,    0],
       [   0,   15,    0,    0,    0,    0,    0, 1055,    0,    0],
       [   0,   22,    0,    1,    0,    0,    0,  921,    0,    0],
       [   0,    6,    2,    0,    0,    0,    2,  968,    0,    0]])

In [ ]:
#Try cosine complete

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='cosine', linkage='complete')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.3853


array([[ 952,    1,    0,    0,    0,    0,   47,    1,    0,    0],
       [   0, 1115,    0,    0,    0,    0,    8,    4,    0,    0],
       [  59,  231,    0,    0,    0,    1,  684,   16,    0,    0],
       [ 169,  647,    0,    0,    0,    0,  183,   33,    0,    0],
       [  53,   27,    0,    0,    0,   39,  241,  620,    0,    0],
       [ 152,  478,    0,    0,    0,   96,   60,   77,    0,    0],
       [ 117,    9,    0,    0,    0,    1,  753,  134,    0,    0],
       [  48,   42,    0,    0,    0,    3,   40,  937,    0,    0],
       [ 124,  557,    0,    0,    0,   15,  130,  118,    0,    0],
       [  41,   59,    0,    0,    0,    0,  192,  686,    0,    0]])

In [ ]:
#Try cosine single

# Compute hierarchical clustering
numClusters = 10
Hclustering = AgglomerativeClustering(n_clusters=numClusters, affinity='cosine', linkage='single')
Hclustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = Hclustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.1136


array([[   0, 1001,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1127,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  990,    1,    0,    0,    0,    0,    0,    0,    0],
       [   0, 1032,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  980,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,  859,    0,    0,    0,    4,    0,    0,    0,    0],
       [   0, 1013,    0,    0,    0,    0,    1,    0,    0,    0],
       [   0, 1069,    0,    0,    0,    0,    0,    1,    0,    0],
       [   0,  943,    0,    0,    0,    0,    0,    0,    1,    0],
       [   0,  977,    0,    0,    0,    0,    0,    0,    0,    1]])

## Part 2: Exploring an unsupervised method of your choice

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import sklearn
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale

import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import MeanShift

In [ ]:
#Get required data
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#Create a batch of training data of size 10,000
X_train = np.asarray(train_images.reshape(train_images.shape[0],-1))
X_batch=X_train[0:10000,:]

#Create a batch of the corresponding correct labels of size 10,000
y = train_labels
y_prec = train_labels[0:10000]

In [ ]:
#mapCluster method given to us by Dr. Pauca

def mapCluster(y_clusters, y, numClusters):
  y_pred = np.zeros(y_clusters.shape).astype(np.int32)
  mapOfCluster = np.zeros(numClusters).astype(np.int32) * -1
  #Map clusters to digits in mapOfClusters by choosing the digit most often associated with each cluster
  for cluster in range(0, numClusters):
    #Count number of true labels for each member in this cluster
    count = np.bincount(y[y_clusters == cluster], minlength=10)
    #Map cluster to the true label of most cluster members
    #Pick the index of the first max valuein count if there are more than max values
    mapOfCluster[cluster] = np.argmax(count)

    # print(cluster, "map to", y_pred[cluster], ", counts: ", count)

  #Now use mapOfCluster to map each y_cluster to predicted class
  y_pred = mapOfCluster[y_clusters]
  #This is the loop version of the line above
  # for i in range(0, len(y_clusters)):
    #     y_pred[i] = mapOfCluster[y_clusters[i]]

  return y_pred

In [ ]:
#Try K-Means

#Create a batch of training data of size 10,000
X_train = np.asarray(train_images.reshape(train_images.shape[0],-1))
X_batch=X_train[0:10000,:]

# Compute hierarchical clustering
numClusters = 10
clustering = KMeans(n_clusters = numClusters, random_state= 1234)
clustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = clustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.5677


array([[ 787,    5,    8,   44,   18,    0,   40,    3,   96,    0],
       [   0, 1119,    2,    1,    1,    0,    0,    2,    2,    0],
       [   7,  181,  662,   61,   22,    0,   23,   11,   24,    0],
       [   6,  100,   30,  601,   11,    0,    9,   35,  240,    0],
       [   0,   67,    4,    0,  378,    0,   13,  517,    1,    0],
       [   9,  256,    2,  234,   28,    0,   22,   80,  232,    0],
       [  14,  121,    9,    5,   98,    0,  754,    0,   13,    0],
       [   1,   86,    4,    1,  105,    0,    0,  872,    1,    0],
       [   4,  118,    8,  209,   30,    0,   18,   53,  504,    0],
       [   5,   34,    0,   17,  239,    0,    3,  678,    2,    0]])

In [ ]:
#Try K-Means

# Compute hierarchical clustering
numClusters = 10
clustering = KMeans(n_clusters = numClusters, random_state= 5)
clustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = clustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.5675


array([[ 786,    5,    7,   47,   16,    0,   40,    3,   97,    0],
       [   0, 1119,    2,    1,    1,    0,    0,    2,    2,    0],
       [   7,  180,  664,   60,   22,    0,   23,   11,   24,    0],
       [   6,   99,   30,  597,   12,    0,    9,   34,  245,    0],
       [   0,   66,    4,    0,  377,    0,   14,  519,    0,    0],
       [   9,  255,    2,  233,   29,    0,   22,   81,  232,    0],
       [  14,  120,    9,    5,   96,    0,  757,    0,   13,    0],
       [   1,   87,    4,    1,  105,    0,    0,  871,    1,    0],
       [   4,  118,    8,  207,   31,    0,   18,   54,  504,    0],
       [   5,   34,    0,   17,  241,    0,    3,  676,    2,    0]])

In [ ]:
#Try K-Means with random-state = 5, max_iter = batch size

# Compute hierarchical clustering
numClusters = 10
clustering = KMeans(n_clusters = numClusters, random_state= 5, max_iter= 10000)
clustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = clustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.5675


array([[ 786,    5,    7,   47,   16,    0,   40,    3,   97,    0],
       [   0, 1119,    2,    1,    1,    0,    0,    2,    2,    0],
       [   7,  180,  664,   60,   22,    0,   23,   11,   24,    0],
       [   6,   99,   30,  597,   12,    0,    9,   34,  245,    0],
       [   0,   66,    4,    0,  377,    0,   14,  519,    0,    0],
       [   9,  255,    2,  233,   29,    0,   22,   81,  232,    0],
       [  14,  120,    9,    5,   96,    0,  757,    0,   13,    0],
       [   1,   87,    4,    1,  105,    0,    0,  871,    1,    0],
       [   4,  118,    8,  207,   31,    0,   18,   54,  504,    0],
       [   5,   34,    0,   17,  241,    0,    3,  676,    2,    0]])

In [ ]:
#Try K-Means with random-state = 5, max_iter = batch size

# Compute hierarchical clustering
numClusters = 10
clustering = KMeans(n_clusters = numClusters, random_state= 5, algorithm="full")
clustering.fit(X_batch)

# map the clusters to their digit of maximum frequency in the MNIST dataset
clusters = clustering.labels_
y_pred = mapCluster(clusters, y_prec, numClusters)
print(sm.accuracy_score(y_pred, y_prec))
confusion_matrix(y_prec, y_pred)

0.5675


array([[ 786,    5,    7,   47,   16,    0,   40,    3,   97,    0],
       [   0, 1119,    2,    1,    1,    0,    0,    2,    2,    0],
       [   7,  180,  664,   60,   22,    0,   23,   11,   24,    0],
       [   6,   99,   30,  597,   12,    0,    9,   34,  245,    0],
       [   0,   66,    4,    0,  377,    0,   14,  519,    0,    0],
       [   9,  255,    2,  233,   29,    0,   22,   81,  232,    0],
       [  14,  120,    9,    5,   96,    0,  757,    0,   13,    0],
       [   1,   87,    4,    1,  105,    0,    0,  871,    1,    0],
       [   4,  118,    8,  207,   31,    0,   18,   54,  504,    0],
       [   5,   34,    0,   17,  241,    0,    3,  676,    2,    0]])

# Analysis


---

# 1. Background
Clustering is an unsupervised machine learning technique where data points are compared using a distance metric and grouped into clusters. These groups feature similar data points and can be extremely useful when no labels have been provided. There are many different clustering algorithms, all with their strengths and weaknesses. 5 of the most common are K-Means, Density-Based Spatial Clustering of Applications with Noise (DBSCAN), Mean-Shift Clustering, Expectation-Maximization (EM) Clustering via Gaussian Mixture Models (GMM), and Hierarchical Clustering. Hierarchical Clustering and K-Means Clustering will be evaluated and tested for this project.

**1.1.	Hierarchical Clustering**
Hierarchical Clustering is a connectivity method using the idea that a distance metric can indicate the relation between samples. In other words, closer samples are more similar than farther samples. Hierarchical clustering houses two different methods in it: agglomerative (bottom-up) or divisive (top-down). In agglomerative clustering, each sample exists in its cluster, and clusters (samples) are merged as the algorithm proceeds. The earlier the samples are merged, the more similar they are. In divisive clustering, all samples start in the same cluster, and samples are split into clusters as the algorithm proceeds. Both types can be represented as a dendrogram or a tree-like graph that represents the clusters that exist within the data as well as their respective similarities (with the closer pairs having links closer to the leaves in Agglomerative). There are two key parameters for hierarchical clustering models: a distance metric and linkage type. The distance metric tells the algorithm how to compute the distance between two samples, with different metrics yielding different results based on different underlying equations. Some examples of distance metrics are Euclidean distance, Manhattan distance, and Cosine. Linkage criteria give the algorithm the metric used to merge/unmerge the samples. Some examples are ward, complete, average, and single. Each dataset is different, so the parameters that will maximize the accuracy of the model (yield the best predictions) will vary and should be investigated.

**1.2.	K-Means Clustering**
K-Means clustering is a centroid-based method, using a fixed number of clusters input initially. The clusters are initially placed randomly by using random samples, and each subsequent sample is placed into the cluster closest to it via a distance metric. Once all samples are assigned to a cluster, the algorithm finds a center location of each of these clusters such that the squared distances from each cluster center to samples are minimized. If a sample is now closer to another cluster center, it will move to that new cluster. This process is repeated until a new cluster center yields no reassignment of samples to new clusters. The number of clusters input into the algorithm is fixed, serving as a drawback for this method.  Additionally, initial cluster assignments are random, so each run of the algorithm can produce different results. 

# 2. Methodology
To compare these two techniques and determine the best parameters for each, the same dataset was used. MNIST is a dataset containing images of hand-drawn numbers (0 – 9) with corresponding labels. These images (28 x 28) have been transformed into size (784, 1) where each entry is the intensity of that corresponding pixel. In this case, the number of clusters is known, allowing us to use K-Means clustering. The size of this MNIST dataset is 60,000, but training a model on a dataset this large is impractical and does not produce a significantly better result than a smaller piece. A smaller subset of 10,000 images from the MNIST dataset was selected. 6 sets of 10,000 images were tested to see if any significant difference in accuracy could be achieved by using a specific subset. The first 10,000 images produced the highest accuracy score and were chosen. The relative frequency of each number was calculated via the corresponding training labels, and the first 10,000 images used are representative of the 60,000-image dataset.

When using agglomerative clustering, the number of clusters remained constant at 10 across all tests. Different affinity (distance functions) and linkage were used to determine the maximum accuracy score. The affinity parameters tested were Euclidean, Manhattan, and Cosine and the linkage parameters tested were Ward, Average, Complete, and Single. All combinations were tried, yielding 10 total tests. The algorithm is not aware of which number (0-9) it is grouping, so it places each into a random cluster. A method was used to correctly map the clusters returned by the model back to its “true” label. In other words, the algorithm could map the image 9 into group 0, so the method mapped that 9 from group 0 to 9. An accuracy score (number of correct groupings / total number of samples) was then calculated by comparing the returned clusters and the correct labels given by the MNIST data. A confusion matrix was also calculated to indicate which numbers the model had trouble grouping.

Similarly, for K-Means clustering, the number of clusters remained fixed at 10 throughout all tests. Different random state, max_iter, and algorithm parameters were investigated to determine the optimum values. Random_state gives a random number generation for the initialization of the centroids used as the center of the clusters. An integer passed into this parameter makes the randomness deterministic. 5 and 1234 were used for this value. Max_iter tells the algorithm the maximum number of iterations the k-means algorithm should use on a single run. This number is 300 by default but was moved up to 10,000 (the size of the batch of images used). The algorithm parameter can move between “full” and “Elkan”. Elkan is used by default but full was investigated as well. Similarly, the same method was used to map the clusters returned by the model to their true labels. From there, an accuracy score and confusion matrix were calculated to determine the best parameters and best accuracy of the technique. 


# 3. Results

The accuracy scores and confusion matrices of both techniques (using optimal parameters) along with their different parameters combination results are below. 

Agglomerative Clustering:
The optimal parameters to produce the highest accuracy score received during tests were affinity = Euclidean and linkage = Ward. This combination yielded:

Accuracy Score: 0.6958
Confusion Matrix:
array([[ 979,    1,    1,    6,    2,    0,    9,    0,    3,    0],
       [   0, 1088,   12,    5,    1,    0,    0,   19,    2,    0],
       [  13,    2,  885,    9,    2,    0,    1,   66,   13,    0],
       [   1,    0,    9,  961,    5,    0,    1,   32,   23,    0],
       [   0,    1,    2,    0,  470,    0,   16,  491,    0,    0],
       [   2,    0,    3,  277,    3,    0,    9,  323,  246,    0],
       [  10,    1,    1,   13,    2,    0,  978,    1,    8,    0],
       [   0,    3,    2,    7,   40,    0,    0, 1018,    0,    0],
       [   2,    4,    3,  279,   11,    0,    6,   60,  579,    0],
       [   1,    1,    2,   20,  395,    0,    2,  555,    2,    0]])

The following are the resulting accuracy scores from all tested parameters. For more details and their respective confusion matrices, see the Jupityer Notebook.

Parameters (Affinity, Linkage)	Accuracy Score
Euclidean, Ward	0.6958
Euclidean, Average	0.2297
Euclidean, Complete	0.4109
Euclidean, Single	0.1136
Manhattan, Average	0.2287
Manhattan, Complete	0.3649
Manhattan, Single	0.1136
Cosine, Average	0.2222
Cosine, Complete	0.3853
Cosine, Single	0.1136

K-Means Clustering:
The optimal parameters to produce the highest accuracy score received during tests were Random_state = 1234. This parameter yielded:

Accuracy Score: 0.5677

Confusion Matrix: 
array([[1536,    5,   10,   88,   47,    0,   67,    3,  232,    6],
       [   0, 2265,    3,    3,    1,    0,    3,    1,    4,    1],
       [  19,  269, 1329,  111,   73,    0,   47,   17,   57,    7],
       [  10,  190,   60, 1327,   27,    0,   17,   12,  387,   46],
       [   1,  118,    6,    0,  731,    0,   28,  542,    5,  514],
       [  25,  344,    4,  581,   52,    0,   43,   43,  564,  119],
       [  28,  169,   17,   11,  216,    0, 1488,    0,   42,    0],
       [   1,  152,   15,    1,  205,    0,    0,  955,    2,  762],
       [   7,  228,   11,  469,   53,    0,   18,   67,  989,   80],
       [   8,   73,    1,   39,  489,    0,    4,  587,   12,  801]])

The following are the resulting accuracy scores from tested parameters. 

Parameters (Random State, Max_Iter, Algorithm)	
1234, Default (300), Default (Elkan)	0.5677
5, Default, Default	0.5675
5, 10000, Default	0.5675
5, Default, Full	0.5675

# 4. Analysis and Discussion
For Agglomerative Clustering, the highest accuracy score obtained out of the tested parameters (with Euclidean affinity, Ward linkage) was 0.6958. In other words, out of the 10,000 samples, the algorithm correctly grouped 6,958. This is a high percentage relative to the other parameters tested, but it does leave room for improvement. The confusion matrix highlights this, suggesting that the algorithm had trouble grouping some numbers. The algorithm did not correctly group 5 or 9. 5s were predominantly placed into either 3, 7, or 8 with 9s being predominantly placed into 4 and 7. Additionally, more 4s were placed into 6 than 4. This confusion matrix highlights patterns of error with agglomerative clustering and offers room for improvement with more fine-tuning of parameters or the use of another clustering method. Additionally, parameters matter a lot when attempting to optimize Agglomerative Clustering. The difference between the accuracy scores of the best and worst parameters was 0.5822. This makes sense because of the importance of distance to compare samples. The linkage tells the algorithm which distance to use between observations, merging samples that have the lowest distance. The affinity tells the algorithm how to calculate this distance, serving an important metric for comparison. By changing the distance used and how the distance is calculated, the samples clustered together will change, thus impacting the accuracy of the algorithm. 

For K-Means clustering, the highest accuracy score obtained out of the tested parameters (with Random_state = 1234) was 0.5677. In other words, out of the 10,000 samples, the algorithm correctly grouped 5,677. This accuracy was lower than Agglomerative Clustering, suggesting Agglomerative Clustering is better for this application. However, K-Means beats out Agglomerative Clustering in runtime complexity (O(n^2)) vs (O(n^3)). The confusion matrix associated with these optimal K-Means parameters also points out some patterns of errors. Once again, the algorithm did not correctly group 5 or 9. 9s were grouped into 4 and 7 while 5s were grouped into 2, 4, and 8. Additionally, the different parameters used by K-Means do not impact the accuracy of the model as shown in the table above. With the Random_state constant, a change in Max_Iter and Algorithm did not produce any increase in accuracy. 

Both methods point out a key drawback of unsupervised learning. Despite unsupervised learning being a step up over no technique at all, unsupervised learning can struggle when there are patterns or resemblances between samples in different clusters. For example, digits 3, 5, and 8 can often be confused due to a similar shape. Unsupervised methods receive no feedback from labels, so the model does not learn to distinguish between these digits while training.
